<a href="https://colab.research.google.com/github/Vekram1/QR_algo_visualization/blob/main/QR_algo_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# QR algo implemenataion
def qr_algorithm(A, max_iters=30, tol=1e-2):
    A_k = A.copy()
    A_list = [A_k.copy()]
    Q_total_list = [np.eye(A.shape[0])]
    for _ in range(max_iters):
        Q, R = np.linalg.qr(A_k)
        A_k = R @ Q
        A_list.append(A_k.copy())

        Q_total = Q_total_list[-1] @ Q
        Q_total_list.append(Q_total)

        # Stop if nearly triangular
        if np.allclose(A_k - np.diag(np.diag(A_k)), 0, atol=tol):
            break
    return A_list, Q_total_list



def align_Q_to_eigenpairs(Q_list, eigvecs, eigvals):
    Qc = []
    for Q in Q_list:
        Q2 = Q.copy()
        for j in range(Q.shape[1]):
            v = eigvecs[:, j] * eigvals[j]  # scaled eigenvector
            if np.dot(Q2[:, j], v) < 0:
                Q2[:, j] *= -1.0
        Qc.append(Q2)
    return Qc


def make_Q_continuous(Q_list, ref_vecs):
    Qc = [Q_list[0].copy()]
    for k in range(1, len(Q_list)):
        Q = Q_list[k].copy()
        Q_prev = Qc[-1]
        for j in range(Q.shape[1]):
            if np.dot(Q[:, j], ref_vecs[:, j]) < 0:
                Q[:, j] *= -1.0
            if np.dot(Q[:, j], Q_prev[:, j]) < 0:
                Q[:, j] *= -1.0
        Qc.append(Q)
    return Qc


def animate_qr(A, filename="qr_iteration.gif"):
    A_list, Q_list = qr_algorithm(A, max_iters=40)

    eigvals, eigvecs = np.linalg.eig(A)
    eigvecs = eigvecs / np.linalg.norm(eigvecs, axis=0)


    ref_vecs = eigvecs * eigvals

    Q_list = align_Q_to_eigenpairs(Q_list, eigvecs, eigvals)


    Q_list = make_Q_continuous(Q_list, ref_vecs)
    fig, (ax_mat, ax_vec) = plt.subplots(1, 2, figsize=(10, 5))

    # --- left subplot: matrix entries
    mat_im = ax_mat.imshow(A_list[0], cmap="Greys", vmin=-5, vmax=5)
    ax_mat.set_title("Matrix $A_k$")
    ax_mat.set_xlim(-0.5, A.shape[1]-0.5)
    ax_mat.set_ylim(A.shape[0]-0.5, -0.5)  # flip y to look natural

    # gridlines
    for x in range(A.shape[1]+1):
        ax_mat.axvline(x-0.5, color="black", linewidth=1)
    for y in range(A.shape[0]+1):
        ax_mat.axhline(y-0.5, color="black", linewidth=1)

    ax_mat.set_xticks([])
    ax_mat.set_yticks([])
    for spine in ax_mat.spines.values():
        spine.set_visible(False)

    # numbers
    text_entries = []
    for i in range(A.shape[0]):
        row = []
        for j in range(A.shape[1]):
            t = ax_mat.text(j, i, f"{A_list[0][i,j]:.2f}",
                            ha="center", va="center", color="black", fontsize=14)
            row.append(t)
        text_entries.append(row)

    # right subplot: eigenvector directions
    ax_vec.set_xlim(-1.2, 1.2)
    ax_vec.set_ylim(-1.2, 1.2)
    ax_vec.axhline(0, color="gray", lw=0.5)
    ax_vec.axvline(0, color="gray", lw=0.5)
    ax_vec.set_aspect("equal")
    ax_vec.set_title("Eigenvector directions")

    true_lines = []
    for j in range(eigvecs.shape[1]):
        v = eigvecs[:, j]
        v = v / np.linalg.norm(v)
        line, = ax_vec.plot([0, v[0]], [0, v[1]], 'k--', lw=1.5,
                            label=f"$\\lambda={eigvals[j]:.2f}$")
        true_lines.append(line)

    arrows = [ax_vec.arrow(0, 0, Q_list[0][0,j], Q_list[0][1,j],
                           head_width=0.08, color="C"+str(j), alpha=0.8)
              for j in range(A.shape[0])]

    ax_vec.legend()


    def update(frame):
        # update matrix numbers
        mat_im.set_array(A_list[frame])
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                text_entries[i][j].set_text(f"{A_list[frame][i,j]:.2f}")

        # update QR arrows
        for j in range(A.shape[0]):
            arrows[j].remove()
            v = Q_list[frame][:, j]
            print(v)
            arrows[j] = ax_vec.arrow(0, 0, v[0], v[1],
                                     head_width=0.08, color="C"+str(j), alpha=0.8)
        return [mat_im] + [t for row in text_entries for t in row] + arrows

    ani = animation.FuncAnimation(fig, update, frames=len(A_list),
                                  interval=800, blit=False, repeat=True)
    ani.save(filename, writer="pillow")
    plt.close(fig)


if __name__ == "__main__":
    A = np.array([[2, 1],
                  [1, -4]], dtype=float)
    animate_qr(A, "qr_iteration.gif")


[1. 0.]
[-0. -1.]
[1. 0.]
[-0. -1.]
[0.89442719 0.4472136 ]
[ 0.4472136  -0.89442719]
[ 0.92847669 -0.37139068]
[-0.37139068 -0.92847669]
[0.52409743 0.85165832]
[ 0.85165832 -0.52409743]
[-0.55031336  0.8349582 ]
[0.8349582  0.55031336]
[0.06813398 0.99767618]
[ 0.99767618 -0.06813398]
[-0.27771077  0.96066473]
[0.96066473 0.27771077]
[-0.09787892  0.99519833]
[0.99519833 0.09787892]
[-0.19234515  0.98132734]
[0.98132734 0.19234515]
[-0.14339977  0.98966485]
[0.98966485 0.14339977]
[-0.16888332  0.98563605]
[0.98563605 0.16888332]
[-0.15565704  0.98781116]
[0.98781116 0.15565704]
[-0.16253175  0.98670331]
[0.98670331 0.16253175]
[-0.15896133  0.98728481]
[0.98728481 0.15896133]
Animation saved to qr_iteration.gif
